# Colab Setup

In [1]:
# Install cmudict, not included in colab env
!pip install cmudict

In [0]:
# Authenticate Google Drive Access
# Comment out if not running this on colab
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
count_syllables = drive.CreateFile({'id':'1CGKlsOz9C12RcatJT6iheJrW0Tq7dPvL'})
count_syllables.GetContentFile('count_syllables.py')

missing_words = drive.CreateFile({'id':'1IoXlh3r7-eXip3UPA4hEoHtscZJZ7som'})
missing_words.GetContentFile('missing_words.json')

train = drive.CreateFile({'id':'1L-xLounIWPWjJx78D_26HGubVvyGEDOO'})
train.GetContentFile('train.txt')

# Import packages

In [ ]:
from count_syllables import count_syllables
import random
import ipywidgets as widgets
from IPython.display import display

# Widget for user input

In [5]:
# Colab does not render the widget
# Default option is to generate the full haiku

style = {'description_width': 'initial'}
regen = widgets.Dropdown(options=['full haiku', 'line 2 only', 'line 3 only'],
                 value = 'full haiku',
                 description = 'What would you like to generate?',
                 disabled = False,
                 style=style)

display(regen)

Dropdown(description='What would you like to generate?', options=('full haiku', 'line 2 only', 'line 3 only'),…

# Load training corpus and create Markov chains

In [0]:
# Load a training-corpus text file
with open('train.txt', 'r') as f:
    train = f.read()

# Process the training corpus for spaces, newline breaks, and so on
punc = '!,.?~:\"'
#train = train.translate(str.maketrans('', '', punc))
train_list = train.split()

# Map each word in corpus to the word after (Markov model order 1)
train_dict_o1 = dict()

# Map each word pair in corpus to the word after (Markov model order 2)
train_dict_o2 = dict()

# Populate both corpus dictionaries simultaneously
for first, second, third in zip(train_list, train_list[1:], train_list[2:]):
    if first in train_dict_o1:
        train_dict_o1[first].append(second)
    else:
        train_dict_o1[first] = [second]
    if (first, second) in train_dict_o2:
            train_dict_o2[first, second].append(third)
    else:
        train_dict_o2[first, second] = [third]

# Define some utility functions

In [0]:
def rand_o1():
    '''
    randomly picks a word from the order 1 Markov chain keys
    '''
    word = random.choice(list(train_dict_o1.keys())) 
    return word

def rand_o2():
    '''
    generate a word by randomly picking a key from the order 2 Markov chain
    '''
    seed_word = random.choice(list(train_dict_o2.keys()))
    mapped_word = random.choice(train_dict_o2[seed_word])
    return mapped_word

def follow_word(sentence = ['we', 'saw'], target = 5, syl = 2):
    '''
    generate followings words using the order 2 markov chain
    '''
    while syl < target:
        seed_word = tuple(sentence)
        
        try:
            mapped_word = random.choice(train_dict_o2[seed_word])
        except KeyError: 
            mapped_word = rand_o1()
            
        mapped_word_syl = count_syllables(mapped_word)
        
        if syl + mapped_word_syl <= target:
            sentence.append(mapped_word)
            syl += mapped_word_syl
        else:
            mapped_word = rand_o2()
            mapped_word_syl = count_syllables(mapped_word)
            
            if syl + mapped_word_syl <= target:
                sentence.append(mapped_word)  
                syl += mapped_word_syl
                
    print(syl) 
    sentence = " ".join(sentence)
    return(sentence)

def new_line1():
    '''
    generates first line of the haiku
    '''
    target = 5
    syl= target + 1
    
    # Pick a random first word that's fewer than 5 syllables from the order 1 markov chain
    while syl > target:
        first_word = rand_o1()
        
        # Generate the second word using the order 1 markov chain
        try:
            second_word = random.choice(train_dict_o1[first_word])
        except KeyError: 
            second_word = rand_o1()
        
        syl = count_syllables(first_word) + count_syllables(second_word)
        
    sentence1 = [first_word, second_word]
    
    # Generate followings words using the order 2 markov chain
    sentence1 = follow_word(sentence1)
    return sentence1

def new_line23(prev_sen = 'deep autumn water hill', target = 5):
    '''
    generates second or third line of the haiku
    '''
    syl = target + 1
    
    # Take the last two words from the previous sentence as the seed words for the order 2 markov chain
    seed_word = tuple(prev_sen.split()[-2::])
    while syl > target:

        try:
            first_word = random.choice(train_dict_o2[seed_word])
        except KeyError: 
            first_word = rand_o1()
            
        seed_word = tuple([prev_sen[-1], first_word])
        
        try:
            second_word = random.choice(train_dict_o2[seed_word])
        except KeyError: 
            second_word = rand_o1()
            
        syl = count_syllables(first_word) + count_syllables(second_word) 
        
    sentence = [first_word, second_word] 
    
    # Generate followings words using the order 2 markov chain
    sentence = follow_word(sentence, target, syl)
    return sentence

# Let's build the haiku!

In [9]:
line = regen.value

# Colab does not render the widget
# Default option is to generate the full haiku
# If you wish to generate line 2 or 3 only comment out the aline above and uncomment the following corresponding lines

# line = 'line 2 only'
# line = 'line 3 only'

def haiku(line):
    '''
    generates the haiku based on user selection
    '''
    
    # Define starter sentences

    sentence1 = "round lumps of cells grows"
    sentence2 = "up to love porridge later"
    sentence3 = "become The Supremes"
    
    if line == 'full haiku':
        sentence1 = new_line1()
        sentence2 = new_line23(sentence1, 7)
        sentence3 = new_line23(sentence2, 5)
        
    if line == 'line 2 only':
        sentence2 = new_line23(sentence1, 7)
        
    if line == 'line 3 only':
        sentence3 = new_line23(sentence2, 5)
        
    for x in [sentence1, sentence2, sentence3]:
        print(x)

haiku(line)

5
7
5
utter end wander end
once hollow noons swordhand hill's
fence dancing banks sticks
